In [109]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [110]:
data = pd.read_csv('biddings.csv')
dfo = pd.DataFrame(data)


In [111]:
df = dfo.iloc[:150000, ] 
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,convert
149995,0.06,-5.78,-3.07,-0.84,0.73,0.51,-0.55,0.83,-0.42,1.28,...,-0.03,-0.99,0.25,-0.27,-0.98,0.06,-0.06,-0.60,0.20,0
149996,-0.02,2.08,-0.68,-0.36,0.38,-6.42,1.95,-4.18,5.41,-0.12,...,-0.10,0.72,0.37,-0.04,-0.18,-0.02,0.20,0.13,0.73,0
149997,-0.02,0.86,2.42,-0.20,-1.03,0.02,0.26,0.25,0.50,0.45,...,-0.17,0.71,0.46,-0.03,-0.16,-0.02,-1.31,-0.02,0.02,0
149998,-0.01,-0.26,-2.34,-0.92,1.12,0.45,0.76,1.18,-0.67,1.46,...,0.05,-0.18,-0.09,-0.05,-0.69,-0.05,-0.02,0.01,0.00,0
149999,0.02,-9.37,-4.55,0.23,-1.33,-1.90,1.55,-0.44,1.85,-0.64,...,-0.01,1.13,0.07,1.54,0.87,-1.46,-0.01,1.31,-0.48,0


In [112]:
#Convert column = shown ad was clicked: 1 = clicked ad
df.convert.value_counts()

0    149705
1       295
Name: convert, dtype: int64

### Severely underbalanced

Converted accounts for 1908/999999 = .191% of our data

We will have to oversample or undersample our target. Most likely undersample to minimize our noise

I will use RandomUnderSampler, NearMiss, and EditedNearestNeighbors
to underersample our target here first to get baselines

After doing that the accuracy of models was a bit lower than desirable, I will try a different resampling technique. That is SMOTEEN which combines under/over sampling

In [100]:
from sklearn.model_selection import train_test_split

x = df.drop(['convert'],axis=1) #training features
y = df.convert #target

#Need to set our train/test values before resampling
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                  test_size = .1)

In [101]:
from collections import Counter
from imblearn.combine import SMOTEENN

#Combine over and undersampling
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek()
X_resampled, y_resampled = smote_tomek.fit_resample(x_train, y_train)
print(sorted(Counter(y_resampled).items()))

[(0, 134729), (1, 134729)]


In [114]:
#First model using our undersampled/oversampled data (SMOTEEN)
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

clf = LinearSVC()
clf.fit(X_resampled, y_resampled) 

print('Training accuracy LinearSVC is: {}'.format(clf.score(X_resampled, y_resampled)))
print('Test accuracy LinearSVC is: {}'.format(clf.score(x_test, y_test)))

y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

# scores = cross_val_score(clf, X_resampled, y_resampled, cv=5)
# print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Training accuracy LinearSVC is: 0.6901595053774614
Test accuracy LinearSVC is: 0.6469333333333334
              precision    recall  f1-score   support

           0       1.00      0.65      0.79     14976
           1       0.00      0.58      0.01        24

   micro avg       0.65      0.65      0.65     15000
   macro avg       0.50      0.62      0.40     15000
weighted avg       1.00      0.65      0.78     15000



In [104]:
from sklearn import ensemble

params = {'n_estimators': 600,
          'max_features': 'log2',
          'max_depth': 6}

rfc = ensemble.RandomForestClassifier(**params)
rfc.fit(X_resampled, y_resampled)

print('Training accuracy normal RandomForest is: {}'.format(rfc.score(X_resampled, y_resampled)))
print('Test accuracy normal RandomForest is: {}'.format(rfc.score(x_test, y_test)))

y_pred = rfc.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

# scores = cross_val_score(rfc, X_resampled, y_resampled, cv=5)
# print("Cross Val RandomForestClassifier Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Training accuracy normal RandomForest is: 0.8909366209205145
Test accuracy normal RandomForest is: 0.8282666666666667


KeyboardInterrupt: 

In [107]:
params = {'n_estimators': 500,
          'max_features': 'sqrt',
          'max_depth': 4}

clf1 = ensemble.GradientBoostingClassifier(**params)
clf1.fit(X_resampled, y_resampled)

print('Training accuracy normal GradientBoosting is: {}'.format(clf1.score(X_resampled, y_resampled)))
print('Test accuracy normal GradientBoosting is: {}'.format(clf1.score(x_test, y_test)))

y_pred = clf1.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

# scores = cross_val_score(clf1, X_resampled, y_resampled, cv=5)
# print("Cross Val GradientBoostingClassifier Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Training accuracy normal GradientBoosting is: 0.9989571658662946
Test accuracy normal GradientBoosting is: 0.9965333333333334


In [93]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [400,500,600],
    'max_features': ['sqrt','log2'],
    'max_depth' : [4,5,6]
}

CV_clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
CV_clf.fit(X_resampled, y_resampled)

CV_clf.best_params_

KeyboardInterrupt: 

In [108]:
from sklearn.metrics import classification_report

y_pred = clf1.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14976
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00     15000
   macro avg       0.50      0.50      0.50     15000
weighted avg       1.00      1.00      1.00     15000

